# PURPOSE OF THIS NOTEBOOK

This is V2 of the data manipulation for the BERT model. Instead of trying to predict the exact ONET code, I will be using it to predict the major group the job belongs too. Then again try to predict the job code it belongs too once it predicts the major group it belongs too. (**SOURCE ONET DATA HERE**)

In [7]:
import pandas as pd
import numpy as np
import csv

In [8]:
# Start by loading in the data. 
onet_df = pd.read_csv("../Data/Updated_ONET_Alt_Titles.csv")
base_onet_df = pd.read_csv("../Data/Occupation Data.csv")

# Filter out any columns we don't need. 
onet_filtered = onet_df[['O*NET-SOC Code', 'Reported Job Title']]

# Show the filtered data
print(onet_filtered.shape)
onet_filtered.head()

(62306, 2)


,O*NET-SOC Code,Reported Job Title
0,11-1011.00,Chief Diversity Officer (CDO)
1,11-1011.00,Chief Executive Officer (CEO)
2,11-1011.00,Chief Financial Officer (CFO)
3,11-1011.00,Chief Nursing Officer
4,11-1011.00,Chief Operating Officer (COO)


In [9]:
# remove the military onet codes. 
base_onet_df = base_onet_df[~base_onet_df['O*NET-SOC Code'].str[:2].isin(['55'])]
onet_filtered = onet_filtered[~onet_filtered['O*NET-SOC Code'].str[:2].isin(['55'])]

## Start to strip the codes except for the first two, the major group. 

In [10]:
base_onet_df['O*NET-SOC Group'] = base_onet_df['O*NET-SOC Code'].str[:2]
onet_filtered['O*NET-SOC Group'] = onet_filtered['O*NET-SOC Code'].str[:2]

## Dummify the data and grab the labels each group belongs too

In [11]:
# Dummify the data. 
dummy_onets = pd.get_dummies(onet_filtered['O*NET-SOC Group'])

dummy_onets['Reported Job'] = onet_filtered['Reported Job Title']

In [12]:
dummy_onets.head()

,11,13,15,17,19,21,23,25,27,29,...,37,39,41,43,45,47,49,51,53,Reported Job
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Diversity Officer (CDO)
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Executive Officer (CEO)
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Financial Officer (CFO)
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Nursing Officer
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Operating Officer (COO)


In [13]:
base_onet_df

,O*NET-SOC Code,Title,O*NET-SOC Group
0,11-1011.00,Chief Executives,11
1,11-1011.03,Chief Sustainability Officers,11
2,11-1021.00,General and Operations Managers,11
3,11-1031.00,Legislators,11
4,11-2011.00,Advertising and Promotions Managers,11
...,...,...,...
992,53-7072.00,"Pump Operators, Except Wellhead Pumpers",53
993,53-7073.00,Wellhead Pumpers,53
994,53-7081.00,Refuse and Recyclable Material Collectors,53
995,53-7121.00,"Tank Car, Truck, and Ship Loaders",53


In [15]:
# Do the same to the base ONET codes 
base_onet_df = base_onet_df[['O*NET-SOC Group', 'Title']]

dummy_base = pd.get_dummies(base_onet_df['O*NET-SOC Group'])

In [16]:
dummy_base['Reported Job'] = base_onet_df['Title']

In [17]:
dummy_base.head()

,11,13,15,17,19,21,23,25,27,29,...,37,39,41,43,45,47,49,51,53,Reported Job
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Executives
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Sustainability Officers
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,General and Operations Managers
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Legislators
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Advertising and Promotions Managers


In [18]:
# Add in default onet titles 
dummy_onets = pd.concat([dummy_base, dummy_onets], ignore_index=True)

# replace NaNs with False
dummy_onets.fillna(False, inplace=True)

In [19]:
# Seperate the labels and the reported jobs
label_df = dummy_onets.loc[:,dummy_onets.columns != 'Reported Job']
reported_df = dummy_onets.loc[:, dummy_onets.columns == 'Reported Job']

In [20]:
# Check that we isloated just the labels 
label_df.head()

,11,13,15,17,19,21,23,25,27,29,...,35,37,39,41,43,45,47,49,51,53
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
# export the label dataframe 
label_df.to_csv('../Data/label_df.csv', index=False)

In [22]:
# Check that we islated the reported jobs 
reported_df.head()

,Reported Job
0,Chief Executives
1,Chief Sustainability Officers
2,General and Operations Managers
3,Legislators
4,Advertising and Promotions Managers


In [23]:
# Lastly take the labels and change them to booleans
label_df = label_df.astype(int)

In [24]:
# Confirm that there are 0 null values in the label or reported jobs data 
print(label_df.isna().sum().sum())
print(reported_df.isna().sum().sum())

0
0


In [25]:
# Turn the labels into lists 
label_df['Label'] = label_df.values.tolist()

In [26]:
# Do a train test split on the data. 
def tt_split(df):
    ''' 1. Define the final dataframes that we need     
        2. Create a for loop that will iterate through all of the rows and do a 70/30 split on the data. 
        3. Append the new data to the final dataframes
        4. Reset the index of the final dataframes 
        5. Export the final dataframes. 
        STRETCH GOAL: take this function and convert the dataframe equations to numpy array equations, will be much faster.  ''' 

    # Define the final dataframes 
    test_df = pd.DataFrame(columns=['Reported_Jobs', 'Label'])
    train_df = pd.DataFrame(columns=['Reported_Jobs', 'Label'])

    # Grab all the columns apart from the final reported job title
    label_df = df.drop(columns=['Reported_Jobs', 'Label'])
    label_list = label_df.columns.to_list()
    
    for onet in label_list:
        filter_df = df[df[onet] == 1]
        temp_train_df = filter_df.sample(frac=.7,random_state=150)
        temp_test_df = filter_df.drop(temp_train_df.index).reset_index(drop=True)
        temp_train_df.reset_index(inplace=True, drop=True)

        # Append the new data to the final train/test dataframes 
        train_df = pd.concat([train_df, temp_train_df], ignore_index=True)
        test_df = pd.concat([test_df, temp_test_df], ignore_index=True)

    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    
    train_df = train_df[['Reported_Jobs', 'Label']]
    test_df = test_df[['Reported_Jobs', 'Label']]

    return train_df, test_df

In [27]:
# Create the final dataframe to export
label_df['Reported_Jobs'] = reported_df['Reported Job']


In [28]:
train_df, test_df = tt_split(label_df)

In [29]:
train_df.shape

(43869, 2)

In [30]:
test_df.shape

(18802, 2)

In [31]:
train_df

,Reported_Jobs,Label
0,"Farmers, Ranchers, and Other Agricultural Mana...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Supply Chain Planning Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Sales Vice President (Sales VP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Special Programs Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Offshore Wind Operations Manager,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
43864,Sailors and Marine Oilers,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43865,Dinkey Engine Fireman,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43866,Airveyor Operator,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43867,Process Operator,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
# Export the Training and Testing data. 
train_df.to_csv('../Data/Training_Data.csv', index=False)
test_df.to_csv('../Data/TestingData.csv', index=False)

## Create Data for 22 models. 

Start Data manipulation on spliting based on the major ONET group. Train test split such data and automate export of data for training and model evaluation. 

In [33]:
# Start with the dummified data. 
dummy_onets

,11,13,15,17,19,21,23,25,27,29,...,37,39,41,43,45,47,49,51,53,Reported Job
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Executives
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chief Sustainability Officers
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,General and Operations Managers
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Legislators
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Advertising and Promotions Managers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62666,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,Windlasser
62667,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,Wire Wheeler
62668,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,Yarn Man
62669,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,Yarn Worker


In [36]:
# Start with grabbing a list of all the unique ONET groups. 
onet_group_list = dummy_onets.columns.to_list()

# remove the reported jobs. 
onet_group_list.pop(22)

# confirm that we have all the ONET Groups. 
onet_group_list

['11',
 '13',
 '15',
 '17',
 '19',
 '21',
 '23',
 '25',
 '27',
 '29',
 '31',
 '33',
 '35',
 '37',
 '39',
 '41',
 '43',
 '45',
 '47',
 '49',
 '51',
 '53']

## GOAL: 

I want to be able to split the ONET groups into individual datasets where I can do a train test split on all. This will be taxing on the computer but this can be done. 
1. Loop through the list of ONET Major Groups 
2. isolate into a new subgroup
3. preform the train/test split on that sub data. Already wrote the function so we can easily just run that function
4. export the data and save into a new file

### NOTE: For the tt_split function to work I would also need the reported_jobs column. so it would need to include that  



In [73]:
# import the orginal version of the ONET data again and remerge the data. 
onet_df = pd.read_csv("../Data/Updated_ONET_Alt_Titles.csv")
base_onet_df = pd.read_csv("../Data/Occupation Data.csv")

# Filter out any columns we don't need. 
onet_filtered = onet_df[['O*NET-SOC Code', 'Reported Job Title']]
base_onet_df.rename(columns={'Title':'Reported Job Title'}, inplace=True)

# remove the military onet codes. 
base_onet_df = base_onet_df[~base_onet_df['O*NET-SOC Code'].str[:2].isin(['55'])]
onet_filtered = onet_filtered[~onet_filtered['O*NET-SOC Code'].str[:2].isin(['55'])]

In [74]:
# Add the standardized job titles to the mix. 
onet_filtered = pd.concat([onet_filtered, base_onet_df]).reset_index(drop=True)

In [75]:
# isolate just the 
onet_filtered['ONET_Group'] = onet_filtered['O*NET-SOC Code'].str[:2]

In [113]:
def labelization(onet_filtered, group):
    # Start the base code for the loop where we seperate the data and get it ready for the train test split. 
    isolated_group_df = onet_filtered[onet_filtered['ONET_Group'] == group].drop('ONET_Group', axis=1)

    # after seperating dummify the data to start the labelization. 
    isolated_dummy = pd.get_dummies(isolated_group_df['O*NET-SOC Code'])
    isolated_dummy['Reported_Jobs'] = isolated_group_df['Reported Job Title']

    # Now seperate the reported jobs so we can turn the data into a list
    isolated_label = isolated_dummy.loc[:,isolated_dummy.columns != 'Reported_Jobs'].astype(int)
    isolated_jobs = isolated_dummy.loc[:,isolated_dummy.columns == 'Reported_Jobs']

    # Now turn the label dataframe into a list
    isolated_label['Label'] = isolated_label.values.tolist()
    isolated_label['Reported_Jobs'] = isolated_jobs['Reported_Jobs']

    return isolated_label

In [114]:
onet_group_list = onet_filtered['ONET_Group'].unique().tolist()

In [115]:
for group in onet_group_list:
    # Seperate to the group
    isolated_label = labelization(onet_filtered, group)

    # get the training and testing data 
    train_df, test_df = tt_split(isolated_label)

    # save to a new path
    train_df.to_csv(f'../Data/MajorGroupTrainTestSplit/ONET_{group}_train_df.csv', index=False)
    test_df.to_csv(f'../Data/MajorGroupTrainTestSplit/ONET_{group}_test_df.csv', index=False)


In [117]:
# Export the list 
np.savetxt('../Data/ONET_Group_List.csv', onet_group_list, delimiter=',', fmt='%s', header= 'ONET_Group')